In [49]:
from typing import Optional

# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel

import nest_asyncio
from pyngrok import ngrok
import uvicorn
import json

In [50]:
with open('account.json', 'r') as fp:
    account = json.load(fp)
print(account)

{'abc': '1234', 'bcd': '2345', 'cdef': '3456', 'efg': '4567'}


In [51]:
with open('pet_num.json', 'r') as fp:
    data = json.load(fp)
print(data)

{'abc': 30, 'bcd': 5, 'cdef': 35, 'efg': 43}


In [52]:
def pet_sum(user_id:str, pet_num:int):
  if user_id in list(data.keys()):
    data[user_id]+= pet_num
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]
  else:
    new_m_p = {user_id : pet_num}
    data.update(new_m_p)
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]

In [53]:
# ocr로 입력해서 받는다고 가정한다 
def input_id():
    user_id = 'abc'
    return user_id

In [54]:
from fastapi import FastAPI
from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from core.config import settings
from apis.general_pages.route_homepage import general_pages_router
from fastapi.responses import HTMLResponse

templates = Jinja2Templates(directory="templates")

def include_router(app):
    app.include_router(general_pages_router)


def configure_static(app):  #new
    app.mount("/static", StaticFiles(directory="static"), name="static")


def start_application():
    app = FastAPI(title=settings.PROJECT_NAME,version=settings.PROJECT_VERSION)
    include_router(app)
    configure_static(app) #new
    return app

app = start_application()

# 웹페이지만 나옴 
@app.get('/login/')
def login(request: Request):
    return templates.TemplateResponse("auth/login.html", {"request": request})

# 로그인 결과 출력 
@app.get('/login/result')
def info(request: Request, id:str, pw:str):
    if id in account.keys():
        if account[id] == pw:
            print('pass')
            return {f'{id}님 성공적으로 로그인하셨습니다.'}
        else:
            print('fail')
            return{'비밀번호를 잘못 입력하셨습니다. 다시 로그인해주세요.'}
    else:
      return {'회원가입 후 시도해주세요.'}

# 페트병 개수 
@app.get('/pet/', response_class=HTMLResponse)
async def pet(request: Request, pet_num:int):
    user_id = input_id()
    result = pet_sum(user_id, pet_num)
    img = qrcode.make(f'{ngrok_tunnel.public_url}/pet/?pet_num={pet_num}')
    img.save('result.png')
    return templates.TemplateResponse("main.html", {"request": request, 'id':user_id, 'pet_num':result})


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://ae7b-114-203-149-236.ngrok.io


INFO:     Started server process [7343]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56893 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56893 - "GET /static/images/nolabel.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:56893 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56893 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56894 - "GET /pet/?pet_num=5 HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /static/images/nolabel.png HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /static/images/pet.png HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET / HTTP/1.1" 200 OK
